1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [97]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [98]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [99]:
tashkeel = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
arabicCharacters = "ىعظحرسيشضقثلصطكآماإهزءأفؤغجئدةخوبذتن"
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = [".", "،", ":", "؛", "-", "؟"]
validCharacters= tashkeel + list(arabicCharacters) + punctuations
charcters_without_tashkeel = list(arabicCharacters) + punctuations

In [100]:
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))

<class 'str'>


In [101]:
print(len(f))
# arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
#regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
processedData = re.findall(characters_regex,f)
processedData = " ".join(processedData)
processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
# processedData = ' '.join(word for word in processedData.split() if word not in arabic_stopwords) # remove stopwors from text
print(len(processedData))

18515271
17997403


In [102]:
w = open('processed.txt', 'w', encoding='utf-8')

w.write(processedData)
w.close()

In [103]:
#remove diacritics
without_diacritics= araby.strip_diacritics(processedData)
print(len(without_diacritics))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)
w2.close()

10711561


In [104]:
#take the procrssed text and separate it into sentences
fileRead=open('processed.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longStringSplited=re.sub(r"\n", ' ', longString)
#the dot itself got removed in the process
longStringSplited=longStringSplited.split('.')
print(len(longStringSplited))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)

w3 = open('processed_sentences_separated.txt', 'w', encoding='utf-8')
for line in longStringSplited:
 w3.write(line)

 #now longStringSplited is the list of procrssed text without practices and numbers and dots 
 #the rest of the punctuation still there  


 


40836


In [105]:
# now we want to create a list of words and a crossbonding list of punctuation for each word 

#first remove all puncturation 

fileRead=open('processed_sentences_separated.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longString=re.sub(r"\n", ' ', longString)
punctuations = ["،", ":", "؛", "-", "؟"]
for element in punctuations:
     longString=re.sub(element, '', longString)



In [106]:
# now the varable called longString has a single string with all the processed words in it 
listOfwordsWith_Diacritics=list()
listOfwordsWith_NoDiacritics=list()


without_diacritics= araby.strip_diacritics(longString)


without_diacritics=re.sub(r"\s+", ' ', without_diacritics)
listOfwordsWith_NoDiacritics=without_diacritics.split(" ")



listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")





In [107]:
#now that we have two separated lists we need to get the diacritics list 


listofDiacritrcs_ToWord=list()
temp=list()
counter=0
flag=0
for word in listOfwordsWith_Diacritics:
    while counter<len(word):
     if word[counter] in arabicDictionary: #checking if the character is a letter
      if (counter+1)<len(word):
        #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
        if word[counter +1] in arabicDictionary: 
          temp.append("")
          counter+=1
          continue
      counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
      continue
     else:
      if (counter+1)<len(word):
        if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
          temp.append(word[counter]+word[counter+1])
          counter+=2
          continue
      temp.append(word[counter]) #if the current character only is the diacritics add it to the list
      counter+=1    
    listofDiacritrcs_ToWord.append(temp.copy())     
    temp.clear() 
    counter=0
          
          
    
        


print(listOfwordsWith_Diacritics[0:100])
print(listOfwordsWith_NoDiacritics[0:100])

['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ', 'مَالِكٍ', 'وَأَصْحَابِهِ', 'أَنَّ', 'السَّاحِرَ', 'كَافِرٌ', 'بِاَللَّهِ', 'تَعَالَى', 'قَالَ', 'مَالِكٌ', 'هُوَ', 'كَالزِّنْدِيقِ', 'إذَا', 'عَمِلَ', 'السِّحْرَ', 'بِنَفْسِهِ', 'قُتِلَ', 'وَلَمْ', 'يُسْتَتَبْ', 'قَوْلُهُ', 'لِعَدَمِ', 'مَا', 'تَتَعَلَّقُ', 'إلَخْ', 'أَيْ', 'الْوَصِيَّةُ', 'قَوْلُهُ', 'مَا', 'مَرَّ', 'أَيْ', 'قُبَيْلَ', 'قَوْلِ', 'الْمَتْنِ', 'لَغَتْ', 'وَلَوْ', 'اقْت

In [108]:
print(len(listOfwordsWith_NoDiacritics))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2102046
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

2-Building the LSTM model for character level classifier

In [109]:
for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
# print(listofDiacritrcs_ToWord[0:100])
Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object)
print(str(X_train.shape))

(1, 2102046)


In [110]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each sentence.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [111]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 2102046, 100)      5000000   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 2102046, 100)      0         
 alDropout1D)                                                    
                                                                 
 lstm_32 (LSTM)              (None, 100)               80400     
                                                                 
 dense_28 (Dense)            (None, 4)                 404       
                                                                 
Total params: 5080804 (19.38 MB)
Trainable params: 5080804 (19.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [112]:
# epochs = 5
# batch_size = 64

# history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [113]:
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Dense
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

# # Assuming you have a list of Arabic words and their diacritics
# arabic_words =listOfwordsWith_NoDiacritics [0:50000] # List of Arabic words
# diacritics = listofDiacritrcs_ToWord[0:50000] # List of corresponding diacritics

# # Tokenization
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(arabic_words)
# word_sequences = tokenizer.texts_to_sequences(arabic_words)
# diacritic_sequences = tokenizer.texts_to_sequences(diacritics)

# # Padding
# max_sequence_length = max(len(seq) for seq in word_sequences)
# word_sequences = pad_sequences(word_sequences, maxlen=max_sequence_length)
# diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_sequence_length)

# # Model
# model = Sequential()
# model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=max_sequence_length))
# model.add(LSTM(100, return_sequences=True))
# model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

# # Compile the model
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Training
# model.fit(word_sequences, diacritic_sequences, epochs=10, batch_size=32, validation_split=0.2)




In [114]:
# # Inference
# new_word = "العب"
# input_sequence = tokenizer.texts_to_sequences([new_word])
# padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length)

# # Predict diacritics
# predicted_probabilities = model.predict(padded_sequence)
# predicted_diacritics = predicted_probabilities.argmax(axis=-1)

# # Decode diacritics and combine with the input word
# # Decode diacritics and combine with the input word
# decoded_diacritics = ''.join([tokenizer.index_word.get(int(idx), '') for idx in predicted_diacritics])
# word_with_diacritics = new_word + decoded_diacritics
# print("Word with Diacritics:", word_with_diacritics)
# # Print raw predicted probabilities
# print("Raw Predicted Probabilities:", predicted_probabilities)
# # Print tokenizer's index_word mapping
# print("Tokenizer Index to Word Mapping:", tokenizer.index_word)
# decoded_diacritics = ' '.join([tokenizer.index_word.get(int(idx), '') for idx in predicted_diacritics[0]])
# print("Diacritics:", decoded_diacritics)



In [116]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense



# Tokenize the input words and diacritics
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
word_tokenizer.fit_on_texts(listOfwordsWith_NoDiacritics[0:50000])
word_sequences = word_tokenizer.texts_to_sequences(listOfwordsWith_NoDiacritics[0:50000])

diacritic_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
diacritic_tokenizer.fit_on_texts([''.join(d) for d in listofDiacritrcs_ToWord[0:50000]])
diacritic_sequences = diacritic_tokenizer.texts_to_sequences([''.join(d) for d in listofDiacritrcs_ToWord[0:50000]])

# Pad sequences to have the same length
max_len = max(max(len(seq) for seq in word_sequences), max(len(seq) for seq in diacritic_sequences))
padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len, padding='post')

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_word_sequences, np.expand_dims(padded_diacritic_sequences, -1), epochs=10, batch_size=32)

# Save the model for later use
model.save("diacritic_prediction_model.h5")

# Now, you can use the trained model to predict diacritics for new Arabic words
def predict_diacritics(model, word):
    word_sequence = word_tokenizer.texts_to_sequences([word])
    padded_word_sequence = pad_sequences(word_sequence, maxlen=max_len, padding='post')
    predicted_diacritic_sequence = model.predict(padded_word_sequence)
    predicted_diacritic_sequence = np.argmax(predicted_diacritic_sequence, axis=-1)
    predicted_diacritic = diacritic_tokenizer.sequences_to_texts(predicted_diacritic_sequence)
    return predicted_diacritic[0]




Epoch 1/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.4741 - accuracy: 0.8463
Epoch 2/10
1563/1563 [==============================] - 24s 16ms/step - loss: 0.2503 - accuracy: 0.9140
Epoch 3/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.2073 - accuracy: 0.9291
Epoch 4/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1856 - accuracy: 0.9354
Epoch 5/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1714 - accuracy: 0.9394
Epoch 6/10
1563/1563 [==============================] - 24s 16ms/step - loss: 0.1618 - accuracy: 0.9422
Epoch 7/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1540 - accuracy: 0.9444
Epoch 8/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1477 - accuracy: 0.9464
Epoch 9/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.1426 - accuracy: 0.9481
Epoch 10/10
1563/1563 [==============================] - 24s 16m

In [179]:
# Example usage
input_word = "ذهب علي إلى الشاطئ"
input_word=input_word.split(' ')

for word in input_word: 
  predicted_diacritics1 = predict_diacritics(model, word)
  print(f"Input word: {word}")
  print(f"Predicted diacritics: {predicted_diacritics1}")
  tempString=str()
  counterWord=0
  counterDiacrtic=0
  while counterDiacrtic<(len(predicted_diacritics1)):
    if predicted_diacritics1[counterDiacrtic]!=' ':
      tempString=tempString+word[counterWord]
      tempString=tempString+predicted_diacritics1[counterDiacrtic]
      print(predicted_diacritics1[counterDiacrtic])
      if predicted_diacritics1[counterDiacrtic] == 'ّ':
        print("here")
        tempString=tempString+predicted_diacritics1[counterDiacrtic+2]
        counterDiacrtic=counterDiacrtic+2
      counterWord+=1
    counterDiacrtic+=1

  print(tempString)

1/1 [==============================] - 0s 33ms/step
Input word: ذهب
Predicted diacritics: َ   َ   َ
َ
َ
َ
ذَهَبَ
1/1 [==============================] - 0s 33ms/step
Input word: علي
Predicted diacritics: َ   َ   ّ َ
َ
َ
ّ
here
عَلَيّ 
1/1 [==============================] - 0s 31ms/step
Input word: إلى
Predicted diacritics:   َ
َ
إَ
1/1 [==============================] - 0s 28ms/step
Input word: الشاطئ
Predicted diacritics:   ْ ّ َ     ِ   ِ
ْ
ّ
here
ِ
ِ
اْلّ شِاِ
